In [3]:
import cv2
import base64
import requests
import time
from collections import deque
import numpy as np

# Settings
VIDEO_PATH = "Test1.mp4"
OUTPUT_PATH = "smoothed_tracking.mp4"
API_URL = "https://detect.roboflow.com/nfl-object-tracking/4?api_key=RsIHVYfJftsspBQBQPkK&confidence=0.2"
MAX_HISTORY = 5  # Number of past positions to average

# Load video
cap = cv2.VideoCapture(VIDEO_PATH)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# For smoothing
position_history = deque(maxlen=MAX_HISTORY)

frame_num = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Encode frame
    _, buffer = cv2.imencode('.jpg', frame)
    b64 = base64.b64encode(buffer).decode('utf-8')

    # Roboflow detection
    response = requests.post(API_URL, data=b64, headers={"Content-Type": "application/x-www-form-urlencoded"})
    try:
        json_data = response.json()
        predictions = json_data.get("predictions", [])
        print(f"Frame {frame_num} Predictions: {predictions}")
    except Exception as e:
        print(f"❌ Error parsing JSON at frame {frame_num}: {e}")
        predictions = []


    ball_detected = False
    for pred in predictions:
        if pred["class"] == "Ball" and float(pred["confidence"]) > 0.1:
            print("Detected:", pred["class"], "Confidence:", pred["confidence"])
            x, y = int(pred["x"]), int(pred["y"])
            position_history.append((x, y))
            ball_detected = True
            break  # Use only first confident ball detection

    # Smooth using average of last N positions
    if position_history:
        xs, ys = zip(*position_history)
        avg_x = int(np.mean(xs))
        avg_y = int(np.mean(ys))

        # Draw smoothed position
        cv2.circle(frame, (avg_x, avg_y), 12, (0, 255, 255), -1)
        cv2.putText(frame, "Smoothed Ball", (avg_x + 10, avg_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Save frame
    out.write(frame)
    print(f"Processed frame {frame_num}")
    frame_num += 1
    time.sleep(0.1)

# Cleanup
cap.release()
out.release()
print(f"✅ Video saved: {OUTPUT_PATH}")


Frame 0 Predictions: []
Processed frame 0
Frame 1 Predictions: []
Processed frame 1
Frame 2 Predictions: []
Processed frame 2
Frame 3 Predictions: []
Processed frame 3
Frame 4 Predictions: []
Processed frame 4
Frame 5 Predictions: []
Processed frame 5
Frame 6 Predictions: []
Processed frame 6
Frame 7 Predictions: []
Processed frame 7
Frame 8 Predictions: []
Processed frame 8
Frame 9 Predictions: []
Processed frame 9
Frame 10 Predictions: []
Processed frame 10
Frame 11 Predictions: []
Processed frame 11
Frame 12 Predictions: []
Processed frame 12
Frame 13 Predictions: [{'x': 402.5, 'y': 488.5, 'width': 21.0, 'height': 21.0, 'confidence': 0.44068291783332825, 'class': 'Ball', 'class_id': 0, 'detection_id': 'b8b8992f-40d2-4a27-a28b-53b8cbd2ee6f'}]
Detected: Ball Confidence: 0.44068291783332825
Processed frame 13
Frame 14 Predictions: [{'x': 401.0, 'y': 489.0, 'width': 22.0, 'height': 20.0, 'confidence': 0.27725210785865784, 'class': 'Ball', 'class_id': 0, 'detection_id': '4f79963f-165f-4c